# Single Tutorial: Multi-omics factors analysis
MOFA is a factor analysis model that provides a general framework for the integration of multi-omic data sets in an unsupervised fashion.

This tutorial focuses on how to perform mofa in multi-omics like scRNA-seq and scATAC-seq


## Part.1 MOFA Model
In this part, we construct a model of mofa by scRNA-seq and scATAC-seq

In [1]:
import anndata
import Pyomic
rna=anndata.read_h5ad('sample/rna_p_n_raw.h5ad')
atac=anndata.read_h5ad('sample/atac_p_n_raw.h5ad')

In [2]:
rna,atac

(AnnData object with n_obs × n_vars = 1876 × 25596
     obs: 'Type', 'sample',
 AnnData object with n_obs × n_vars = 1876 × 71272
     obs: 'Type', 'sample')

We only need to add anndata to `Pyomic.single.mofa` to construct the base model

In [3]:
test_mofa=Pyomic.single.mofa(omics=[rna,atac],
                             omics_name=['RNA','ATAC'])

In [5]:
test_mofa.mofa_preprocess()
test_mofa.mofa_run()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Successfully loaded view='rna' group='group0' with N=1876 samples and D=25596 features...
Successfully loaded view='atac' group='group0' with N=1876 samples and D=71272 features...




Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Deter

## Part.2 MOFA Analysis
After get the model by mofa, we need to analysis the factor about different omics, we provide some method to do this

### load data

In [1]:
import pandas as pd
import anndata
import scanpy as sc
import Pyomic

In [6]:
rna=anndata.read_h5ad('sample/rna_test.h5ad')

### add value of factor to anndata

In [7]:
rna=Pyomic.single.factor_exact(rna,hdf5_path='sample/MOFA_POS.hdf5')
rna

AnnData object with n_obs × n_vars = 1792 × 3000
    obs: 'Type', 'sample', 'cell_type', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'factor1', 'factor2', 'factor3', 'factor4', 'factor5', 'factor6', 'factor7', 'factor8', 'factor9', 'factor10', 'factor11'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'

### analysis of the correlation between factor and cell type

In [8]:
Pyomic.single.factor_correlation(adata=rna,cluster='cell_type',factor_list=[1,2,3,4,5])

,factor1,factor2,factor3,factor4,factor5
Regulatory T (Treg) cell,0.553421,7.303644,0.323052,0.252761,0.187171
B cell-2,9.078459,5.625337,8.424565,7.768767,0.268084
B cell-3,0.706249,40.881023,2.267589,0.382177,5.123728
Monocyte,0.248902,0.284968,0.147168,0.399443,0.132685
B cell,1.096582,12.407881,0.154599,0.827906,0.495789
B cell-4,0.279002,3.701674,0.487245,0.631280,0.205712
Endothelial cell,NaN,NaN,NaN,NaN,NaN
Natural killer T (NKT) cell-1,0.119266,0.276104,0.111210,0.030626,0.097991
Natural killer cell,1.127901,27.896554,0.235219,0.241705,0.282893
T cell-1,0.704842,3.701813,0.464042,0.396243,0.459828


### Get the gene/feature weights of different factor

In [9]:
Pyomic.single.get_weights(hdf5_path='sample/MOFA_POS.hdf5',view='RNA',factor=1)

,feature,weights,abs_weights,sig
0,b'FAM174B',-2.911107e-10,2.911107e-10,-
1,b'SYNE2',-3.744153e-09,3.744153e-09,-
2,b'LUZP1',-1.840838e-10,1.840838e-10,-
3,b'GGT1',-3.171818e-10,3.171818e-10,-
4,b'LINC02210',-4.588388e-10,4.588388e-10,-
...,...,...,...,...
2995,b'TREM2',4.046089e-11,4.046089e-11,+
2996,b'CPVL',1.353563e-10,1.353563e-10,+
2997,b'MIPEP',-1.360450e-11,1.360450e-11,-
2998,b'INO80E',-4.721143e-09,4.721143e-09,-
